In [155]:
import matplotlib.pyplot as plt
# Turn Interactive Mode off. Only displays plot with plt.show()
plt.ioff()

import pandas as pd
import numpy as np
import json
import os

from collections import defaultdict

In [156]:
DATA_PATH = "./../scripts/output"

In [157]:
lambda_version = "step-v3"

# # Mono : (crop,scaledown,mirror,bw,rotate,watermark)
step_version="step-agg-csmbrw-{}Sec-batch".format(batch_len)
stages_id = ["app"]
stages_suffix = [""]
powerValues = [1024, 2048, 4096, 8192]


# # Manual : crop->scaledown->mirror->bw->rotate->watermark
step_version="step-agg-c-s-m-b-r-w-{}Sec-batch".format(batch_len)
stages_id = ["crop", "scaledown", "mirror", "bw", "rotate", "watermark"]
stages_suffix = ["_cropped", "_resized", "_mirror", "_bw", "_rot", "_watermarked"]
powerValues = [1024, 2048, 4096, 8192]


# # AGG1 : crop->scaledown->(mirror,bw,rotate,watermark)
step_version="step-agg-c-s-mbrw-{}Sec-batch".format(batch_len)
stages_id = ["crop", "scaledown", "mirror_bw_rotate_watermark"]
stages_suffix = ["_cropped", "_resized", "_mirror_bw_rot_watermarked"]
powerValues = [1024, 2048, 4096, 8192]

# # AGG2 : crop->scaledown->(mirror,bw)->(rotate,watermark)
step_version="step-agg-c-s-mb-rw-{}Sec-batch".format(batch_len)
stages_id = ["crop", "scaledown", "mirror_bw", "rotate_watermark"]
stages_suffix = ["_cropped", "_resized", "_mirror_bw", "_rot_watermarked"]
powerValues = [1024, 2048, 4096, 8192]

# # AGG3 : crop->(scaledown,mirror,bw,rotate,watermark)
step_version="step-agg-c-smbrw-{}Sec-batch".format(batch_len)
stages_id = ["crop", "scaledown_mirror_bw_rotate_watermark"]
stages_suffix = ["_cropped", "_resized_mirror_bw_rot_watermarked"]
powerValues = [1024, 2048, 4096, 8192]

# Stage Wise Plots

In [151]:
def plotStageInvocationTimeVsMemory(path, batch_name, step_version, stages_id, displayPlot=False):
    plt.figure(figsize=(10,5))
    for stage in stages_id: 
        f = open('{}/{}/{}/{}.json'.format(path, batch_name, step_version, stage), 'r')
        data = json.load(f)
        power_values = []
        stage_stats = []
        for stat in data['stats']:
            stage_stats.append(stat['averageDuration']/1000)
            power_values.append(stat['value'])
        plt.plot(power_values, stage_stats, label = stage, marker='o')
        f.close()
    plt.legend()
    plt.title(step_version)
    plt.xlabel('Memory(MB)')
    plt.ylabel('Invocation Time(s)')

    # Create names on the x axis
    plt.xticks(power_values)
    
    # Save Plot
    # Create folder if not exists
    if not os.path.exists("./plots/{}/stagewise".format(batch_name)):
        # os.mkdir("./plots/{}".format(batch_name))
        os.mkdir("./plots/{}/stagewise".format(batch_name))
    plt.savefig("./plots/{}/stagewise/InvocationTimeMemoryPlot_{}.png".format(batch_name, step_version))
    if displayPlot:
        plt.show()
    
def plotStageInvocationCostVsMemory(path, batch_name, step_version, stages_id, displayPlot=False):
    plt.figure(figsize=(10,5))
    for stage in stages_id:
        f = open('{}/{}/{}/{}.json'.format(path, batch_name, step_version, stage), 'r')
        data = json.load(f)
        stage_stats = []
        power_values = []
        for stat in data['stats']:
            stage_stats.append(stat['averagePrice']/1000)
            power_values.append(stat['value'])
        plt.plot(power_values, stage_stats, label = stage, marker='o')
        f.close()
    plt.legend()
    plt.title(step_version)
    plt.xlabel('Memory(MB)')
    plt.ylabel('Invocation Cost(USD)')

    # Create names on the x axis
    plt.xticks(power_values)
    
    # Save Plot
    # Create folder if not exists
    if not os.path.exists("./plots/{}/stagewise".format(batch_name)):
        # os.mkdir("./plots/{}".format(batch_name))
        os.mkdir("./plots/{}/stagewise".format(batch_name))
    plt.savefig("./plots/{}/stagewise/InvocationCostMemoryPlot_{}.png".format(batch_name, step_version))
    if displayPlot:
        plt.show()
    
    
def plotStageInvocationTimeCostVsMemory(path, batch_name, step_version, stages_id, displayPlot=False):
    # create figure and axis objects with subplots()
    fig,ax = plt.subplots(figsize=(10,5))
    
    for stage in stages_id: 
        f = open('{}/{}/{}/{}.json'.format(path, batch_name, step_version, stage), 'r')
        data = json.load(f)
        power_values = []
        stage_stats = []
        for stat in data['stats']:
            stage_stats.append(stat['averageDuration']/1000)
            power_values.append(stat['value'])
        ax.plot(power_values, stage_stats, label = stage, marker='o')
        f.close()
    ax.set_xlabel('Memory(MB)')
    ax.set_ylabel('Invocation Time(s)')
        
    ax2=ax.twinx()
    for stage in stages_id:
        f = open('{}/{}/{}/{}.json'.format(path, batch_name, step_version, stage), 'r')
        data = json.load(f)
        stage_stats = []
        power_values = []
        for stat in data['stats']:
            stage_stats.append(stat['averagePrice']/1000)
            power_values.append(stat['value'])
        ax2.plot(power_values, stage_stats, label = stage, marker='x', linestyle="--")
        f.close()
    ax2.set_xlabel('Memory(MB)')
    ax2.set_ylabel('Invocation Cost(USD)')

    # Create names on the x axis
    plt.xticks(power_values)
    plt.legend()
    plt.title(step_version)
    
    # Save Plot
    # Create folder if not exists
    if not os.path.exists("./plots/{}/stagewise".format(batch_name)):
        # os.mkdir("./plots/{}".format(batch_name))
        os.mkdir("./plots/{}/stagewise".format(batch_name))
    plt.savefig("./plots/{}/stagewise/TimeCostMemoryPlot_{}.png".format(batch_name, step_version))
    if displayPlot:
        plt.show()

In [152]:
batch_len = 150
for batch_len in [120, 150, 300]:
    batch_name = "{}Sec".format(batch_len)
    # # Mono : (crop,scaledown,mirror,bw,rotate,watermark)
    step_version="step-agg-csmbrw-{}Sec-batch".format(batch_len)
    stages_id = ["app"]
    plotStageInvocationTimeVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationTimeCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)

    # # Manual : crop->scaledown->mirror->bw->rotate->watermark
    step_version="step-agg-c-s-m-b-r-w-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown", "mirror", "bw", "rotate", "watermark"]
    plotStageInvocationTimeVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationTimeCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)

    # # AGG1 : crop->scaledown->(mirror,bw,rotate,watermark)
    step_version="step-agg-c-s-mbrw-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown", "mirror_bw_rotate_watermark"]
    plotStageInvocationTimeVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationTimeCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)

    # # AGG2 : crop->scaledown->(mirror,bw)->(rotate,watermark)
    step_version="step-agg-c-s-mb-rw-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown", "mirror_bw", "rotate_watermark"]
    plotStageInvocationTimeVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationTimeCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)

    # # AGG3 : crop->(scaledown,mirror,bw,rotate,watermark)
    step_version="step-agg-c-smbrw-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown_mirror_bw_rotate_watermark"]
    plotStageInvocationTimeVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)
    plotStageInvocationTimeCostVsMemory(DATA_PATH, batch_name, step_version, stages_id)

# Cumulative Time Cost Plots for Pipeline Variants

In [142]:
def getPipelineTimeCost(path, batch_name, step_version, stages_id, displayPlot=False):
    total_cost_dict = defaultdict(lambda : 0)
    total_time_dict = defaultdict(lambda : 0)
    for stage in stages_id:
        f = open('{}/{}/{}/{}.json'.format(path, batch_name, step_version, stage), 'r')
        data = json.load(f)
        for stat in data['stats']:
            total_cost_dict[stat["value"]] += stat["averagePrice"] 
            total_time_dict[stat["value"]] += stat['averageDuration']/1000 
        f.close()
    return list(total_cost_dict.keys()), list(total_time_dict.values()), list(total_cost_dict.values())

def plotPipelineTimeCost(path, batch_name, step_version, stages_id, displayPlot=False):
    power_values, total_time, total_cost = getPipelineTimeCost(path, batch_name, step_version, stages_id)
    
    # Plot Time
    fig,ax = plt.subplots(figsize=(10,5))
    ax.plot(power_values, total_time, label="Invocation Time")
    ax.set_xlabel('Memory(MB)')
    ax.set_ylabel('Invocation Time(s)')
    plt.legend()
    
    # Plot Cost
    ax2=ax.twinx()
    ax2.plot(power_values, total_cost, linestyle="--", label="Invocation Cost")
    ax2.set_ylabel('Invocation Cost(USD)')
    
    # Label
    plt.xticks(power_values)
    plt.legend()
    plt.title(step_version)
    
    # Save Plot
    # Create folder if not exists
    if not os.path.exists("./plots/{}/cumulative".format(batch_name)):
        os.mkdir("./plots/{}".format(batch_name))
        os.mkdir("./plots/{}/cumulative".format(batch_name))
    plt.savefig("./plots/{}/cumulative/CumulativeTimeCostMemoryPlot_{}.png".format(batch_name, step_version))
    if displayPlot:
        plt.show()

### Plot

In [144]:
for batch_len in [120, 150, 300]:
    batch_name = "{}Sec".format(batch_len)

    # # Mono : (crop,scaledown,mirror,bw,rotate,watermark)
    step_version="step-agg-csmbrw-{}Sec-batch".format(batch_len)
    stages_id = ["app"]
    plotPipelineTimeCost(DATA_PATH, batch_name, step_version, stages_id)

    # # Manual : crop->scaledown->mirror->bw->rotate->watermark
    step_version="step-agg-c-s-m-b-r-w-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown", "mirror", "bw", "rotate", "watermark"]
    plotPipelineTimeCost(DATA_PATH, batch_name, step_version, stages_id)

    # # AGG1 : crop->scaledown->(mirror,bw,rotate,watermark)
    step_version="step-agg-c-s-mbrw-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown", "mirror_bw_rotate_watermark"]
    plotPipelineTimeCost(DATA_PATH, batch_name, step_version, stages_id)

    # # AGG2 : crop->scaledown->(mirror,bw)->(rotate,watermark)
    step_version="step-agg-c-s-mb-rw-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown", "mirror_bw", "rotate_watermark"]
    plotPipelineTimeCost(DATA_PATH, batch_name, step_version, stages_id)

    # # AGG3 : crop->(scaledown,mirror,bw,rotate,watermark)
    step_version="step-agg-c-smbrw-{}Sec-batch".format(batch_len)
    stages_id = ["crop", "scaledown_mirror_bw_rotate_watermark"]
    plotPipelineTimeCost(DATA_PATH, batch_name, step_version, stages_id)

### Get Data

In [162]:
allStatsDict = {"pipeline":[], "aggregation":[], "batchLen":[], "memory":[], "timePerBatch":[], "costPerBatch":[]}

batch_len = 120
batch_len = 150
batch_len = 300

for batch_len in [120, 150, 300]:
    batch_name = "{}Sec".format(batch_len)

    # # Mono : (crop,scaledown,mirror,bw,rotate,watermark)
    step_pipeline="step-agg-csmbrw-{}Sec-batch".format(batch_len)
    agg_strat = step_pipeline.rstrip("-{}-batch".format(batch_name)).lstrip("step-agg-")
    stages_id = ["app"]
    # Get Stats
    power_values, total_time, total_cost = getPipelineTimeCost(DATA_PATH, batch_name, step_pipeline, stages_id)
    n = len(power_values)
    allStatsDict["pipeline"] += [step_pipeline]*n
    allStatsDict["aggregation"] += [agg_strat]*n
    allStatsDict["batchLen"] += [batch_len]*n
    allStatsDict["memory"] += power_values
    allStatsDict["timePerBatch"] += total_time
    allStatsDict["costPerBatch"] += total_cost

    # # Manual : crop->scaledown->mirror->bw->rotate->watermark
    step_pipeline="step-agg-c-s-m-b-r-w-{}Sec-batch".format(batch_len)
    agg_strat = step_pipeline.rstrip("-{}-batch".format(batch_name)).lstrip("step-agg-")
    stages_id = ["crop", "scaledown", "mirror", "bw", "rotate", "watermark"]
    # Get Stats
    power_values, total_time, total_cost = getPipelineTimeCost(DATA_PATH, batch_name, step_pipeline, stages_id)
    n = len(power_values)
    allStatsDict["pipeline"] += [step_pipeline]*n
    allStatsDict["aggregation"] += [agg_strat]*n
    allStatsDict["batchLen"] += [batch_len]*n
    allStatsDict["memory"] += power_values
    allStatsDict["timePerBatch"] += total_time
    allStatsDict["costPerBatch"] += total_cost

    # # AGG1 : crop->scaledown->(mirror,bw,rotate,watermark)
    step_pipeline="step-agg-c-s-mbrw-{}Sec-batch".format(batch_len)
    agg_strat = step_pipeline.rstrip("-{}-batch".format(batch_name)).lstrip("step-agg-")
    stages_id = ["crop", "scaledown", "mirror_bw_rotate_watermark"]
    # Get Stats
    power_values, total_time, total_cost = getPipelineTimeCost(DATA_PATH, batch_name, step_pipeline, stages_id)
    n = len(power_values)
    allStatsDict["pipeline"] += [step_pipeline]*n
    allStatsDict["aggregation"] += [agg_strat]*n
    allStatsDict["batchLen"] += [batch_len]*n
    allStatsDict["memory"] += power_values
    allStatsDict["timePerBatch"] += total_time
    allStatsDict["costPerBatch"] += total_cost

    # # AGG2 : crop->scaledown->(mirror,bw)->(rotate,watermark)
    step_pipeline="step-agg-c-s-mb-rw-{}Sec-batch".format(batch_len)
    agg_strat = step_pipeline.rstrip("-{}-batch".format(batch_name)).lstrip("step-agg-")
    stages_id = ["crop", "scaledown", "mirror_bw", "rotate_watermark"]
    # Get Stats
    power_values, total_time, total_cost = getPipelineTimeCost(DATA_PATH, batch_name, step_pipeline, stages_id)
    n = len(power_values)
    allStatsDict["pipeline"] += [step_pipeline]*n
    allStatsDict["aggregation"] += [agg_strat]*n
    allStatsDict["batchLen"] += [batch_len]*n
    allStatsDict["memory"] += power_values
    allStatsDict["timePerBatch"] += total_time
    allStatsDict["costPerBatch"] += total_cost

    # # AGG3 : crop->(scaledown,mirror,bw,rotate,watermark)
    step_pipeline="step-agg-c-smbrw-{}Sec-batch".format(batch_len)
    agg_strat = step_pipeline.rstrip("-{}-batch".format(batch_name)).lstrip("step-agg-")
    stages_id = ["crop", "scaledown_mirror_bw_rotate_watermark"]
    # Get Stats
    power_values, total_time, total_cost = getPipelineTimeCost(DATA_PATH, batch_name, step_pipeline, stages_id)
    n = len(power_values)
    allStatsDict["pipeline"] += [step_pipeline]*n
    allStatsDict["aggregation"] += [agg_strat]*n
    allStatsDict["batchLen"] += [batch_len]*n
    allStatsDict["memory"] += power_values
    allStatsDict["timePerBatch"] += total_time
    allStatsDict["costPerBatch"] += total_cost

In [168]:
totalStatDF = pd.DataFrame(allStatsDict)

In [169]:
totalStatDF["noOfBatch"] = totalStatDF.apply(lambda x: 10*60/x["batchLen"], axis=1)

totalStatDF["timeTotal"] = totalStatDF.apply(lambda x: x["timePerBatch"]*x["noOfBatch"], axis=1)

totalStatDF["costTotal"] = totalStatDF.apply(lambda x: x["costPerBatch"]*x["noOfBatch"], axis=1)

totalStatDF.to_csv('./data/Pipeline-Memory-TimeCost.csv')